In [2]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys
import os
import numpy as np
from tqdm import tqdm
import glob
import librosa
import pandas as pd

import torchaudio
from speechbrain.pretrained.interfaces import foreign_class

torchvision is not available - cannot save figures


In [3]:
classifier = foreign_class(source="/mntcephfs/lab_data/shoinoue/Models/trained_models/accent/CustomEncoderWav2vec2Classifier-a72df039c801fa14a1c3226e95ab8c14/", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier", savedir="/mntcephfs/lab_data/shoinoue/Models/trained_models/accent/CustomEncoderWav2vec2Classifier-a72df039c801fa14a1c3226e95ab8c14/", run_opts={"device":"cuda"})
accents = ["us", "england", "australia", "indian", "canada", "bermuda", "scotland", "african", "ireland", "newzealand", "wales", "malaysia", "philippines", "singapore", "hongkong", "southatlandtic",]

def run_classification(path):
    audio, _ = librosa.load(path, sr=fs)
    torchaudio.save(tempfile, torch.tensor(audio).unsqueeze(0), fs)
    signal, org_sr = torchaudio.load(tempfile)
    signal = torchaudio.functional.resample(signal, orig_freq=org_sr, new_freq=fs)
    outputs =  classifier.encode_batch(signal)
    embeddings =  outputs[0].detach().cpu().numpy()
    outputs = classifier.mods.output_mlp(outputs)
    rawaccent = outputs[0].detach().cpu().numpy()
    probaccent = classifier.hparams.softmax(outputs).detach().cpu().numpy()[0]
    return embeddings, rawaccent, probaccent

def cosine_similarity(e1, e2): # from wespeaker, delete the normalizing part
    e1 = torch.tensor(e1)
    e2 = torch.tensor(e2)
    cosine_score = torch.dot(e1, e2) / (torch.norm(e1) * torch.norm(e2))
    cosine_score = cosine_score.item()
    return cosine_score

speechbrain.lobes.models.huggingface_wav2vec - wav2vec 2.0 is frozen.


In [9]:
# mode = "additional_vctk_1hr"
mode = "trial_bigvgan"
base_dir = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/"
accent_dir = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/Hindi/wav/"
native_dir = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/English/wav/"
fs = 16000

labels = {}
if mode=="additional_vctk_1hr":
    labels["VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_smaller"] = "only-gt_small"
    labels["VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small"] = "only-gt_normal"
    labels["VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating"] = "only-gt_large"
    labels["VTN_fine-tuning_nocondition_mix2synVCTK1hr_melmel_hubert_norepeating_smaller"] = "gtsynVCTK_small"
    labels["VTN_fine-tuning_nocondition_mix2synVCTK1hr_melmel_hubert_norepeating_small"] = "gtsynVCTK_normal"
    labels["VTN_fine-tuning_nocondition_mix2synVCTK1hr_melmel_hubert_norepeating"] = "gtsynVCTK_large"
elif mode=="trial_bigvgan":
    labels["VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_smaller"] = "only-gt_small"
    labels["VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small"] = "only-gt_normal"
    labels["VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating"] = "only-gt_large"
    labels["VTN_fine-tuning_nocondition_mix2synVCTK1hr_80mel80mel_hubert_norepeating_smaller"] = "gtsynVCTK_small"
    labels["VTN_fine-tuning_nocondition_mix2synVCTK1hr_80mel80mel_hubert_norepeating_small"] = "gtsynVCTK_normal"
    labels["VTN_fine-tuning_nocondition_mix2synVCTK1hr_80mel80mel_hubert_norepeating"] = "gtsynVCTK_large"
base_dir = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/"

- Save

In [10]:
save = False
la = "Hindi"
tempfile = "temp.wav"

testsamples = list(np.load("data_split_ARCTIC.npy", allow_pickle=True))[2]
for mn in labels:
    print(mn)
    files = glob.glob(base_dir + mn + "/100000/wav/*.wav")
    files.sort()
    for path in tqdm(files):
        basename = os.path.basename(path)[:-4]
        accent = accent_dir + basename + ".wav"
        native = native_dir + basename + ".wav"
        save_path = "/".join(path.split("/")[:-2])+"/accent_embeddings/" + basename + ".npy"
        if not(os.path.exists(save_path)):
        # if True:
            embeddings, rawaccent, probaccent = run_classification(path)
            dirname = os.path.dirname(save_path)
            if not(os.path.exists(dirname)):
                os.makedirs(dirname, exist_ok=True)
            data = {
                "embeddings": embeddings,
                "raw": rawaccent,
                "prob": probaccent
            }
            np.save(save_path, data)
            
        accent_path = f"./evaluation/accent_embeddings/{la}/{basename}.npy"
        native_path = f"./evaluation/accent_embeddings/English/{basename}.npy"
        paths = [accent_path, native_path]
        if not(os.path.exists(accent_path) and os.path.exists(native_path)):
        # if True:
            for i, path in enumerate([accent, native]):
                save_path = paths[i]
                embeddings, rawaccent, probaccent = run_classification(path)
                dirname = os.path.dirname(save_path)
                if not(os.path.exists(dirname)):
                    os.makedirs(dirname, exist_ok=True)
                data = {
                    "embeddings": embeddings,
                    "raw": rawaccent,
                    "prob": probaccent
                }
                np.save(save_path, data)

VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_smaller


100%|██████████| 100/100 [00:00<00:00, 2076.53it/s]


VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small


100%|██████████| 100/100 [00:00<00:00, 5966.29it/s]


VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating


...

100%|██████████| 100/100 [00:00<00:00, 5261.23it/s]


VTN_fine-tuning_nocondition_mix2synVCTK1hr_80mel80mel_hubert_norepeating_small


100%|██████████| 100/100 [00:00<00:00, 4730.29it/s]


VTN_fine-tuning_nocondition_mix2synVCTK1hr_80mel80mel_hubert_norepeating


100%|██████████| 100/100 [00:00<00:00, 4541.01it/s]


- Analysis

In [11]:
save = False
la = "Hindi"
scores = {}
testsamples = list(np.load("data_split_ARCTIC.npy", allow_pickle=True))[2]
for mn in ["Hindi", "English"]:
    scores[mn] = {}
    for key in ["raw", "prob"]:
        scores[mn][key] = []
for mn in labels:
    scores[mn] = {}
    for key in ["embeddings", "raw", "prob"]:
        scores[mn][key] = []
    print(mn)
    files = glob.glob(base_dir + mn + "/100000/wav/*.wav")
    files.sort()
    for path in tqdm(files):
        basename = os.path.basename(path)[:-4]
        if not(basename in testsamples):
            continue
        pred_path = "/".join(path.split("/")[:-2])+"/accent_embeddings/" + basename + ".npy"
        accent_path = f"./evaluation/accent_embeddings/{la}/{basename}.npy"
        native_path = f"./evaluation/accent_embeddings/English/{basename}.npy"
        pred = np.load(pred_path, allow_pickle=True).item()
        accent = np.load(accent_path, allow_pickle=True).item()
        native = np.load(native_path, allow_pickle=True).item()
        
        key = "embeddings"
        a = cosine_similarity(pred[key], accent[key])
        b = cosine_similarity(pred[key], native[key])
        diff = a-b
        scores[mn][key] += [diff]
        
        for key in ["raw", "prob"]:
            scores[mn][key] += [pred[key][3]]
            scores["Hindi"][key] += [accent[key][3]]
            scores["English"][key] += [native[key][3]]

VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_smaller


100%|██████████| 100/100 [00:00<00:00, 619.41it/s]


VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small


100%|██████████| 100/100 [00:00<00:00, 612.05it/s]


VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating


...

100%|██████████| 100/100 [00:00<00:00, 639.15it/s]


VTN_fine-tuning_nocondition_mix2synVCTK1hr_80mel80mel_hubert_norepeating_small


100%|██████████| 100/100 [00:00<00:00, 622.73it/s]


VTN_fine-tuning_nocondition_mix2synVCTK1hr_80mel80mel_hubert_norepeating


100%|██████████| 100/100 [00:00<00:00, 548.24it/s]


In [12]:
for mn in list(labels.keys())+["Hindi", "English"]:
    for key in scores[mn]:
        scores[mn][key] = np.mean(scores[mn][key])
df = pd.DataFrame(scores).T
df = np.round(df, 3)
df.columns = ["classification score (before softmax)", "classification prob.", "AECS ratio"]
df.index = ["Hindi", "English"] + list(labels.values())
df[(1-pd.notna(df)).astype(bool)] = ""
df

,classification score (before softmax),classification prob.,AECS ratio
Hindi,13.955,0.819,
English,-9.042,0.000,
only-gt_small,16.173,0.898,0.466
only-gt_normal,16.507,0.943,0.488
only-gt_large,13.174,0.835,0.436
gtsynVCTK_small,15.984,0.910,0.453
gtsynVCTK_normal,17.595,0.960,0.512
gtsynVCTK_large,15.967,0.903,0.493
